In [12]:

from commonfunctions import *
from Pre_Processing_Functions import *
from skimage.transform import resize
import os
import pickle
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import cv2
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
#Supporting functions

def HoG(image):
    # Resize the image to 130x276
    resized_img = cv2.resize(image, (64, 128))
    # print(resized_img.shape)

    # Convert the original image to gray scale
    resized_img_gray = cv2.cvtColor(resized_img, cv2.COLOR_BGR2GRAY)
    '''
    # Specify the parameters for our HOG descriptor
    win_size = (64, 128)  # You need to set a proper window size
    block_size = (16, 16)
    block_stride = (8, 8)
    cell_size = (8, 8)
    num_bins = 9
    '''
    
    win_size = (64, 128)  # Increase the window size
    block_size = (32, 32)  # Increase the block size
    block_stride = (16, 16)  # Increase the block stride
    cell_size = (16, 16)  # Increase the cell size
    num_bins = 18  # Increase the number of bins

    # Set the parameters of the HOG descriptor using the variables defined above
    hog = cv2.HOGDescriptor(win_size, block_size, block_stride, cell_size, num_bins)

    # Compute the HOG Descriptor for the gray scale image
    hog_descriptor = hog.compute(resized_img_gray)
    return hog_descriptor


def img_pre_processing(image):

    # (1) crop background
    #cropped_image=crop_background(image)

    # (2) Apply Gaussian filtering to smooth out the image and remove any noise that may affect the accuracy of the HOG feature extractor
    #we need to try different kernel sizes for the best!
    gaussian_image = cv2.GaussianBlur(image, (3, 3), 0)

    # Convert the image to floating point
    gaussian_image_float = gaussian_image.astype(np.float32)

    # Calculate the histogram
    hist = cv2.calcHist([image], [0], None, [256], [0,256])

    # Calculate the mean of the histogram
    mean_hist = np.mean(hist)

    # Convert the image to HSV color space
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Extract the S channel
    s_channel = hsv_image[:, :, 1]

    # Calculate the average saturation
    average_saturation = np.mean(s_channel)

    # Decide the gamma value
    if (average_saturation > 128) and (mean_hist > 128):
        gamma = 1.2  # or any value greater than 1
    else:
        gamma = 0.8  # or any value less than 1

    # Apply gamma correction
    gamma_corrected_image = cv2.pow(gaussian_image_float/ 255, gamma)


    final_image_before_HOG=gamma_corrected_image*255
    
    image=final_image_before_HOG.astype(np.uint8)
    
    return image


In [ ]:
#Image editing => Cropping,rotation
def crop_image(image, box):
    x, y, w, h = cv2.boundingRect(box)
    if w < h:
        w, h = h, w
    return image[y:y + h, x:x + w]

def extract_boundaries(image):
    # Read the image
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply GaussianBlur to reduce noise and improve edge detection
    blurred = cv2.GaussianBlur(image, (3, 3), 0)

    # Use the Canny edge detector
    edges = cv2.Canny(blurred, 50, 150)

    return edges

def fill_small_holes(image, kernel_size=3):
    # Read the image
    image = extract_boundaries(image)

    # Apply GaussianBlur to reduce noise
    blurred = cv2.GaussianBlur(image, (3, 3), 0)

    # Apply Canny edge detection
    edges = cv2.Canny(blurred, 50, 150)

    # Define a kernel for morphological operations
    kernel = np.ones((kernel_size, kernel_size), np.uint8)

    # Perform closing to fill small holes
    closed = cv2.morphologyEx(edges, cv2.MORPH_CLOSE, kernel)

    # Perform erosion
    dilated_edges = cv2.dilate(edges, kernel, iterations=1)
    eroded_edges = cv2.erode(dilated_edges, kernel, iterations=1)

    return eroded_edges

def auto_rotate_image(image, counter=1):
    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply GaussianBlur to reduce noise and aid contour detection
    blurred = cv2.GaussianBlur(gray, (3, 3), 0)

    # Use Canny edge detection
    edges = cv2.Canny(blurred, 50, 150)

    # Find contours in the edged image
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    image_with_contour = image.copy()

    # Find the contour with the maximum area
    max_contour = max(contours, key=cv2.contourArea)

    # Fit a rotated bounding box to the contour
    rect = cv2.minAreaRect(max_contour)

    # Unpack the rectangle information
    (center, (width_contour, height_contour), angle) = rect

    # Ensure width is always the larger side
    if width_contour < height_contour:
        width_contour, height_contour = height_contour, width_contour
        angle -= 90

    height, width = image.shape[:2]
    width_ratio = float(width_contour / width)

    #Counter is for the second call of this function, in this call I am interested in getting the new coordinates of the bounding box but no further rotation needed.
    if counter == 0 and width_ratio > 0.65:

        # Rotate the image to the detected angle
        rotated_image = rotating_image(image_with_contour, angle)
    else:
        return image_with_contour, max_contour

    return rotated_image, max_contour

def rotating_image(image, angle):
    # Get the image dimensions
    height, width = image.shape[:2]

    # Calculate the rotation matrix
    rotation_matrix = cv2.getRotationMatrix2D((width / 2, height / 2), angle, 1)

    # Determine the new bounding box after rotation
    cos_theta = np.abs(rotation_matrix[0, 0])
    sin_theta = np.abs(rotation_matrix[0, 1])

    new_width = int((width * cos_theta) + (height * sin_theta))
    new_height = int((width * sin_theta) + (height * cos_theta))

    # Adjust the rotation matrix for translation to keep the entire rotated image
    rotation_matrix[0, 2] += (new_width - width) / 2
    rotation_matrix[1, 2] += (new_height - height) / 2

    # Apply the rotation to the image
    rotated_image = cv2.warpAffine(image, rotation_matrix, (new_width, new_height),
                                    borderMode=cv2.BORDER_CONSTANT, borderValue=(255, 255, 255))

    return rotated_image

#Test function

def Background_Removal():
    images_folder = 'SAR_Background/'
    images = os.listdir(images_folder)

    for image in images:
        img_path = os.path.join(images_folder, image)
        img = cv2.imread(img_path)
        if img is not None:
            new_image, contour1 = auto_rotate_image(img, 0)

            rect1 = cv2.minAreaRect(contour1)
            box1 = cv2.boxPoints(rect1)
            box1 = np.int0(box1)

            # Unpack the rectangle information
            (center1, (width_contour1, height_contour1), angle1) = rect1

            # Ensure width is always the larger side
            if width_contour1 < height_contour1:
                width_contour1, height_contour1 = height_contour1, width_contour1
                angle1 -= 90

            rotated_image, contour2 = auto_rotate_image(new_image)
            rect2 = cv2.minAreaRect(contour2)
            box2 = cv2.boxPoints(rect2)

            (center2, (width_contour2, height_contour2), angle2) = rect2

            # Ensure width is always the larger side
            if width_contour2 < height_contour2:
                width_contour2, height_contour2 = height_contour2, width_contour2
                angle2 -= 90

            if -1 < angle2 < 1 and height_contour2 > 50:
                cropped_image = crop_image(rotated_image, box2)
            elif -1 < angle1 < 1 and height_contour1 > 50:
                new_height, new_width = rotated_image.shape[:2]
                # Resize the image
                resized_image = cv2.resize(new_image, (new_width, new_height))
                cropped_image = crop_image(resized_image, box1)     
            else:
                cropped_image = rotated_image

            show_images([cropped_image, rotated_image, new_image, img],['cropped','rotated','New_image','before'])

        else:
            print(f"Cannot open {image}")

In [14]:
#HOG from scratch!
from pathlib import Path
#from typing import Final

import numpy as np
import skimage as sk


def compute_gradient(img: np.ndarray, grad_filter: np.ndarray) -> np.ndarray:

    ts = grad_filter.shape[0]

    new_img = np.zeros((img.shape[0] + ts - 1, img.shape[1] + ts - 1))

    new_img[int((ts-1)/2.0):img.shape[0] + int((ts-1)/2.0), 
            int((ts-1)/2.0):img.shape[1] + int((ts-1)/2.0)] = img

    result = np.zeros((new_img.shape))
    
    for r in np.uint16(np.arange((ts-1)/2.0, img.shape[0]+(ts-1)/2.0)):
        for c in np.uint16(np.arange((ts-1)/2.0, img.shape[1]+(ts-1)/2.0)):
            curr_region = new_img[r-np.uint16((ts-1)/2.0):r+np.uint16((ts-1)/2.0)+1, 
                                  c-np.uint16((ts-1)/2.0):c+np.uint16((ts-1)/2.0)+1]
            curr_result = curr_region * grad_filter
            score = np.sum(curr_result)
            result[r, c] = score

    result_img = result[np.uint16((ts-1)/2.0):result.shape[0]-np.uint16((ts-1)/2.0), 
                        np.uint16((ts-1)/2.0):result.shape[1]-np.uint16((ts-1)/2.0)]

    return result_img

def compute_gradient_magnitude(horizontal_gradient: np.ndarray, vertical_gradient: np.ndarray) -> np.ndarray:

    mag_img=np.sqrt(np.power(vertical_gradient,2)+np.power(horizontal_gradient,2))
    return mag_img



def compute_gradient_direction(horizontal_gradient: np.ndarray, vertical_gradient: np.ndarray) -> np.ndarray:

    horizontal_gradient+=1e-5
    img_dir=np.rad2deg(np.arctan(vertical_gradient/horizontal_gradient))%180
    return img_dir



def find_nearest_bins(curr_direction: float, hist_bins: np.ndarray) -> (int, int):
 
    min=200
    c=0
    ind_first=10
    ind_second=10
    if (curr_direction>160):
        diff=np.abs(curr_direction-hist_bins[8])
        ind_first=8
        ind_second=0
    else:
        for i in range(9):
            new=np.floor(curr_direction // 10)
            curr_direction=new * 10
            diff=curr_direction-hist_bins[i]
            if (diff<0):
                diff=diff*-1
            if (diff<min):
                min=diff
                c=i
        ind_first=c
        ind_second=c+1

    return (ind_first,ind_second)


def update_histogram_bins(
        HOG_cell_hist: np.ndarray, 
        curr_direction: float, 
        curr_magnitude: float, 
        first_bin_idx: int, 
        second_bin_idx: int, 
        hist_bins: np.ndarray
    ) -> None:

    size=20
    ratiosecond=(hist_bins[second_bin_idx]-curr_direction)/size
    ratiofirst=1-ratiosecond
    HOG_cell_hist[first_bin_idx]+=curr_magnitude*ratiofirst
    HOG_cell_hist[second_bin_idx]+=curr_magnitude*ratiosecond


def calculate_histogram_per_cell(
        cell_direction: np.ndarray, 
        cell_magnitude: np.ndarray, 
        hist_bins: np.ndarray
    ) -> np.ndarray:


    empty_array=np.zeros(hist_bins.shape)
    
    for i in range(8):
        for j in range(8):
            first,second=find_nearest_bins(cell_direction[i,j],hist_bins)
            update_histogram_bins(empty_array,cell_direction[i,j],cell_magnitude[i,j],first,second,hist_bins)
    return empty_array


def compute_hog_features(image):

    image = sk.transform.resize(image, (128, 64))

    if image.shape[2] == 4:
        image = image[:, :, :3]

    if len(image.shape) == 3:
        image = sk.color.rgb2gray(image)

    # Define gradient masks
    horizontal_mask = np.array([-1, 0, 1])
    vertical_mask = np.array([[-1], [0], [1]])

    # Compute gradients
    horizontal_gradient = compute_gradient(image, horizontal_mask)
    vertical_gradient = compute_gradient(image, vertical_mask)

    # Compute gradient magnitude and direction
    grad_magnitude = compute_gradient_magnitude(horizontal_gradient, vertical_gradient)
    grad_direction = compute_gradient_direction(horizontal_gradient, vertical_gradient)

    # Define histogram bins
    hist_bins = np.array([10, 30, 50, 70, 90, 110, 130, 150, 170])

    cells_histogram = np.zeros((16, 8, 9))
    for r in range(0, grad_magnitude.shape[0], 8):
        for c in range(0, grad_magnitude.shape[1], 8):
            cell_direction = grad_direction[r:r+8, c:c+8]
            cell_magnitude = grad_magnitude[r:r+8, c:c+8]
            #calculate histogram per cell
           # cells_histogram[r//8, c//8] = compute_cell_histogram(cell_direction, cell_magnitude, hist_bins)

            cells_histogram[int(r/8), int(c/8)] = calculate_histogram_per_cell(cell_direction, cell_magnitude, hist_bins)

    # Normalize and concatenate histograms
    features_list = []
    for r in range(cells_histogram.shape[0] - 1):
        for c in range(cells_histogram.shape[1] - 1):
            histogram_16x16 = np.reshape(cells_histogram[r:r+2, c:c+2], (36,))
            histogram_16x16_normalized = histogram_16x16 / (np.linalg.norm(histogram_16x16) + 1e-5)
            features_list.append(histogram_16x16_normalized)

    return np.concatenate(features_list, axis=0)



In [ ]:
#SVM from scratch (53%)
class Karim:

    def __init__(self, C = 1.0):
        # C = error term
        self.C = C
        self.w = 0
        self.b = 0
        
       # Hinge Loss Function / Calculation
    def hingeloss(self, w, b, x, y):
        # Regularizer term
        reg = 0.5 * (w * w)

        for i in range(x.shape[0]):
            # Optimization term
            opt_term = y[i] * ((np.dot(w, x[i])) + b)

        # calculating loss
        loss = reg + self.C * max(0, 1 - opt_term)

        return loss[0][0]
    
    def fit(self, X, Y, batch_size=100, learning_rate=0.001, epochs=1000):
        # The number of features in X
        number_of_features = X.shape[1]

        # The number of Samples in X
        number_of_samples = X.shape[0]

        c = self.C

        # Creating ids from 0 to number_of_samples - 1
        ids = np.arange(number_of_samples)

        # Shuffling the samples randomly
        np.random.shuffle(ids)

        # creating an array of zeros
        w = np.zeros((1, number_of_features))
        b = 0
        losses = []

        # Gradient Descent logic
        for i in range(epochs):
            # Calculating the Hinge Loss
            l = self.hingeloss(w, b, X, Y)

            # Appending all losses 
            losses.append(l)
            
            # Starting from 0 to the number of samples with batch_size as interval
            for batch_initial in range(0, number_of_samples, batch_size):
                gradw = 0
                gradb = 0

                for j in range(batch_initial, batch_initial + batch_size):
                    if j < number_of_samples:
                        x = ids[j]
                        ti = Y[x] * (np.dot(w, X[x].T) + b)

                        if ti > 1:
                            gradw += 0
                            gradb += 0
                        else:
                            # Calculating the gradients

                            #w.r.t w 
                            gradw += c * Y[x] * X[x]
                            # w.r.t b
                            gradb += c * Y[x]

                # Updating weights and bias
                w = w + learning_rate * gradw
                b = b + learning_rate * gradb
        
        self.w = w
        self.b = b

        return self.w, self.b, losses  
    
    def predict(self, X):
        
        prediction = np.dot(X, self.w[0]) + self.b # w.x + b
        return np.sign(prediction)
    

In [ ]:
#SVM from scratch (50%)
class Karim2:

    def __init__(self, C = 1.0):
        # C = error term
        self.C = C
        self.w = 0
        self.b = 0
        
    def compute_cost(self,x, y,w,b):
   
        m = x.shape[0] 
        cost = self.C
        

        for i in range(m):
            f_wb = w * x[i] + b
            cost = cost + (f_wb - y[i])**2
        total_cost = 1 / (2 * m) * cost

        return total_cost
    
    def compute_gradient(self,x, y,w,b): 
        # Number of training examples
        m = x.shape[0] 
        cost=0
        dj_dw = 0
        dj_db = 0

        for i in range(m):  
            f_wb = w * x[i] + b 
            dj_dw_i = (f_wb - y[i]) * x[i] 
            dj_db_i = f_wb - y[i] 
            dj_db += dj_db_i
            dj_dw += dj_dw_i 
        dj_dw = dj_dw / m 
        dj_db = dj_db / m 

        return dj_dw, dj_db
    
    def gradient_descent(self,x, y, alpha=0.0001, num_iters=10000): 
        
        w=self.w
        b=self.b

        for i in range(num_iters):
            # Calculate the gradient and update the parameters using gradient_function
            dj_dw, dj_db = self.compute_gradient(x, y, w , b)   
            
            #calculate cost
            if i<100000:      # prevent resource exhaustion 
                cost =  self.compute_cost(x, y, w, b)  

            # Update Parameters using equation (3) above
            b = b - alpha * dj_db                            
            w = w - alpha * dj_dw
            
        self.w=w
        self.b=b
        self.C=cost
    
        return self.w, self.b,self.C
    
    def predict(self,x):
        return np.sign(np.dot(x, self.w[0]) + self.b)

In [20]:
# prepare data
input_dir = 'Full_Dataset/'
#categories = ['1EGP','5EGP','10EGP','20EGP','50EGP','100EGP','200EGP']
#categories = ['1EGP','5EGP','10EGP','20EGP','50EGP','100EGP','200EGP']
#categories = ['1SAR','5SAR','10SAR','50SAR','100SAR','500SAR']
categories = ['1EGP','1SAR', '5EGP','5SAR','10EGP','10SAR','20EGP','50EGP','50SAR','100EGP','100SAR','200EGP','500SAR']

data = []
feature_4 = []

labels = []

for category_idx, category in enumerate(categories):
    for file in os.listdir(os.path.join(input_dir, category)):
        img_path = os.path.join(input_dir, category, file)

        img = io.imread(img_path)
        processed_img=img_pre_processing(img)
        #HOG_class=compute_hog_features(img)
        image=processed_img.astype(np.uint8)
        HOG_class=HoG(image)

        feature_4.append(HOG_class.flatten())

        labels.append(category_idx)


data=np.asarray(feature_4)

labels = np.asarray(labels)


In [21]:
# train / test split
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, shuffle=True, stratify=labels)

In [22]:
# train classifier
classifier = SVC()
#classifier.fit(x_train,y_train)

parameters = [{'gamma': [0.01, 0.001, 0.0001], 'C': [1, 10, 100, 1000]}] #to find the best combination for better results

grid_search = GridSearchCV(classifier, parameters)

grid_search.fit(x_train, y_train)

GridSearchCV(estimator=SVC(),
             param_grid=[{'C': [1, 10, 100, 1000],
                          'gamma': [0.01, 0.001, 0.0001]}])

In [23]:
# test performance
best_estimator = grid_search.best_estimator_

y_prediction = best_estimator.predict(x_test)

score = accuracy_score(y_prediction, y_test)

print('{}% of samples were correctly classified'.format(str(score * 100)))

pickle.dump(best_estimator, open('./model_20fix.p', 'wb'))

99.24528301886792% of samples were correctly classified


In [ ]:
#testing of SVM from scratch (50%)
'''
classifier = Karim2()
classifier.gradient_descent(x_train,y_train)
y_pred = classifier.predict(x_test)
y_prediction=classifier.predict(x_test)
y_prediction=y_prediction[:,0]
print(y_prediction.shape)
score=accuracy_score(y_prediction,y_test)


print('{}% of samples were correctly classified'.format(str(score * 100)))
'''

In [ ]:
#testing SVM from scratch (53%)
'''
from sklearn.preprocessing import StandardScaler
# Scale features
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

# train classifier
classifier = Karim()
classifier.fit(x_train_scaled,y_train)
'''